In [1]:
import pandas as pd
import numpy as np
import os
import quantstats as qs
import json
import pyodbc
from datetime import datetime, timedelta, date

In [ ]:
class SQL:
    def __init__(self, driver, server, database):
        self.driver = driver
        self.server = server
        self.database = database

    def append_table(self, table_name, dataframe):
        try:
            cxn = pyodbc.connect(
                "DRIVER=" + self.driver + ";"
                "SERVER=" + self.server + ";"
                "DATABASE=" + self.database + ";"
                "TRUSTED_CONNECTION=yes;"
            )

            cursor = cxn.cursor()
            columns_query = f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}' ORDER BY COLUMN_NAME"
            cursor.execute(columns_query)
            columns_info = cursor.fetchall()

            table_columns = [column[0] for column in columns_info]
            common_columns = [col for col in dataframe.columns if col in table_columns]
            column_list = ", ".join("[" + column[0] + "]" for column in columns_info)

            placeholders = ", ".join("?" for _ in common_columns)
            query = f"INSERT INTO {table_name} ({column_list}) VALUES ({placeholders})"
            dataframe_subset = dataframe[common_columns]

            prepared_data = []
            for row in dataframe_subset.itertuples(index=False):
                prepared_row = []
                
                for i, value in enumerate(row):
                    column_name = common_columns[i]
                    data_type = next((column[1] for column in columns_info if column[0] == column_name), None)
                    
                    if isinstance(value, pd.Timestamp):
                        value = value.to_pydatetime()
                    
                    elif pd.isna(value):
                        value = None
                    
                    elif data_type == "decimal":
                        value = decimal.Decimal(str(value))
                    
                    elif data_type == "float":
                        value = float(value)
                    
                    elif data_type == "int":
                        value = int(value)
                        
                    else:
                        pass
                    
                    prepared_row.append(value)
                
                prepared_data.append(prepared_row)

            if not prepared_data:
                print("Data to be inserted into StockHistory is empty!")
            
            else:
                pass

            cursor.executemany(query, prepared_data)
            cursor.commit()
            cursor.close()
            cxn.close()
            
        except Exception as e:
            print(f"An error occurred while appending to the table StockHistory: {e}")
            
    def fetch_latest_date(self, table_name):
        try:
            cxn = pyodbc.connect(
                "DRIVER=" + self.driver + ";"
                "SERVER=" + self.server + ";"
                "DATABASE=" + self.database + ";"
                "TRUSTED_CONNECTION=yes;"
            )

            cursor = cxn.cursor()
            query = f"SELECT MAX(current_date) FROM {table_name}"
            cursor.execute(query)
            latest_date = cursor.fetchone()[0]
            cursor.close()
            cxn.close()
            
            return latest_date

class DailyDataDump:
    def __init__(self, config_file_path):
        """
        Initialize StockDataDownloader object.

        Parameters:
        - config_file_path (str): Path to the configuration JSON file.
        """
        # Read JSON file
        self.config_file_path = config_file_path
        with open(config_file_path, encoding="utf-8") as f:
            self.config = json.load(f)
        
        # Initialize an empty DataFrame to store data
        self.all_data = pd.DataFrame()
        
    def download_data(self):
        """
        Download data for each stock symbol and save it into a single CSV file.
        """
        all_stock_data = pd.DataFrame()
        all_other_data = pd.DataFrame()
        
        # Iterate over each stock symbol
        for symbol in self.config["all_stock_symbols"]:
            # Download data using yfinance
            data = yf.download(symbol, period="1d")
            
            # Add the asset name and category column
            data["asset_name"] = symbol
            data["asset_category"] = "Stock"
            
            # Concatenate data to the main DataFrame
            all_stock_data = pd.concat([all_stock_data, data])
        
        # Iterate over other symbols
        for symbol in self.config["all_other_symbols"]:
            # Download data using yfinance
            data = yf.download(symbol, period="1d")
            
            # Add the asset name and category column
            data["asset_name"] = symbol
            
            if symbol in ['SPY', 'MTUM', 'IWN', 'EFA', 'EEM', 'XHB', 'XLB', 'XLE', 'XLY', 'XLK', 'XLV', 'XLI', 'XLU', 'XLP', 'XLF', 'XLC', 'XLRE']:
                asset_category = "ETF"
            elif symbol in ['DBC', 'GLD']:
                symbol = "Gold"
            elif symbol in ['BIL', 'IEF', 'BWX', 'LQD', 'TLT']:
                asset_category = "Treasury"
            elif symbol == 'VNQ':
                asset_category = "REIT"
            
            data["asset_category"] = asset_category
            
            # Concatenate data to the main DataFrame
            all_other_data = pd.concat([all_other_data, data])
            
        self.all_data = pd.concat([all_stock_data, all_other_data])
        
        # Rename columns and drop unnecessary columns
        self.all_data.rename(
            columns={
                "Open": "open_price",
                "High": "high_price",
                "Low": "low_price",
                "Close": "close_price",
                "Volume": "volume",
                "Date": "current_date"
            },
            inplace=True
        )
        
        self.all_data["current_date"] = pd.to_datetime(self.all_data["current_date"], format='%Y-%m-%d %H:%M:%S').dt.date
        self.all_data.drop("Adj Close", axis=1, inplace=True)
        
        return
    
    def calculate_metrics(self):
        self.read_data()
        grouped_data = self.all_data.groupby('asset_name')
        self.all_data["ratio_sharpe"] = self.all_data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sharpe(x["close_price"])))
        self.all_data["ratio_sortino"] = self.all_data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sortino(x["close_price"])))
        self.all_data["ratio_win_loss"] = self.all_data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.win_loss_ratio(x["close_price"])))
        self.all_data["ratio_drawdown"] = self.all_data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.max_drawdown(x["close_price"])))
        
        # Initialize an empty list to store DataFrames
        data_frames = []
        
        # Iterate over each stock symbol
        for file in self.config["all_files"]:
            # Read data from CSV file
            data = pd.read_csv(file)
            
            # Extract asset name from file name
            asset_name = os.path.basename(file).split(".csv")[0]
            
            # Add the symbol column
            if asset_name in ['SPY', 'MTUM', 'IWN', 'EFA', 'EEM', 'XHB', 'XLB', 'XLE', 'XLY', 'XLK', 'XLV', 'XLI', 'XLU', 'XLP', 'XLF', 'XLC', 'XLRE']:
                asset_category = "ETF"
            elif asset_name in ['DBC', 'GLD']:
                asset_category = "Gold"
            elif asset_name in ['BIL', 'IEF', 'BWX', 'LQD', 'TLT']:
                asset_category = "Treasury"
            elif asset_name == 'VNQ':
                asset_category = "REIT"
            else:
                asset_category = "Stock"
            
            # Add asset name, category and date columns
            data["asset_category"] = asset_category
            
            if "Stock" in file:
                data["asset_name"] = data["Symbol"]
            else:
                data["asset_name"] = asset_name
                
            
            # Append DataFrame to the list
            data_frames.append(data)
        
        # Concatenate data frames without resetting index
        old_data = pd.concat(data_frames, ignore_index=True)
        
        # Rename columns and drop unnecessary columns
        old_data.rename(
            columns={
                "Open": "open_price",
                "High": "high_price",
                "Low": "low_price",
                "Close": "close_price",
                "Volume": "volume",
                "Date": "current_date"
            },
            inplace=True
        )
        
        old_data.drop(["Symbol", "Adj Close"], axis=1, inplace=True)
        
        # Filter data to include only the past year's data
        old_data["current_date"] = pd.to_datetime(old_data["current_date"], format='%Y-%m-%d %H:%M:%S').dt.date
        one_year_ago = date.today() - pd.DateOffset(years=1)
        old_data = old_data[old_data["current_date"] >= one_year_ago]
        
        # Concatenate old and current data frames without resetting index
        self.all_data = pd.concat(data_frames, ignore_index=True)
        
        self.all_data.sort_values(["asset_name", "current_date"], inplace=True)
        
        # Calculate percentage returns for different periods
        self.all_data['percentage_1_d_returns'] = grouped_data['close_price'].pct_change(periods=1) * 100
        self.all_data['percentage_1_m_returns'] = grouped_data['close_price'].pct_change(periods=20) * 100
        self.all_data['percentage_3_m_returns'] = grouped_data['close_price'].pct_change(periods=60) * 100
        self.all_data['percentage_1_y_returns'] = grouped_data['close_price'].pct_change(periods=252) * 100
        
        # Calculate percentage volatility for different periods
        self.all_data['percentage_1_m_volatility'] = grouped_data['close_price'].pct_change(periods=20).rolling(window=20).std() * np.sqrt(252) * 100
        self.all_data['percentage_3_m_volatility'] = grouped_data['close_price'].pct_change(periods=60).rolling(window=60).std() * np.sqrt(252) * 100
        self.all_data['percentage_1_y_volatility'] = grouped_data['close_price'].pct_change(periods=252).rolling(window=252).std() * np.sqrt(252) * 100
        
        self.all_data = self.all_data[self.all_data["current_date"] == date.today()]
        
        return
    
    def dump_historical_data(self):
        self.calculate_metrics()
        
        sql = SQL(
            self.config["driver"],
            self.config["server"],
            self.config["database"]
        )
        
        latest_date = sql.fetch_latest_date("end_of_day_asset_details")
        if latest_date < date.today():
            sql.append_table("end_of_day_asset_details", self.all_data)
            print("EOD data dumped successully!")
            
        else:
            print("EOD data already present!")
        
        return

class DailyPortfolioDetails:
    def __init__(self, config_file_path):
        """
        Initialize StockDataDownloader object.

        Parameters:
        - config_file_path (str): Path to the configuration JSON file.
        """
        # Read JSON file
        self.config_file_path = config_file_path
        with open(config_file_path, encoding="utf-8") as f:
            self.config = json.load(f)
        
        # Initialize an empty DataFrame to store data
        self.portfolio_details = pd.DataFrame()
    
    def calculate_metrics_and_insert(self, data):
        # Dummy data for testing
        portfolio_data = [
            ('pan123', 'Portfolio1', 'Stocks', 'AAPL', 40.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '2024-03-30', '2024-03-30'),
            ('pan123', 'Portfolio1', 'Stocks', 'GOOG', 30.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '2024-03-30', '2024-03-30'),
            ('pan123', 'Portfolio1', 'Bonds', 'US Treasuries', 30.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '2024-03-30', '2024-03-30')
        ]
        portfolio_columns = ['pan_card', 'portfolio_name', 'asset_category', 'asset_name', 'percentage_allocation', 'percentage_1_d_cagr', 'percentage_3_m_cagr',
                             'percentage_1_y_cagr', 'percentage_3_m_volatility', 'percentage_1_y_volatility', 'percentage_drawdown',
                             'ratio_sharpe', 'ratio_sortino', 'ratio_win_loss', 'asset_addition_date', 'current_date']
       
        # Create DataFrame from dummy data
        portfolio_df = pd.DataFrame(portfolio_data, columns=portfolio_columns)
       
        # Connect to the database
        conn = self.engine.connect()
       
        # Iterate over rows of portfolio DataFrame
        for index, row in portfolio_df.iterrows():
            asset_addition_date = pd.to_datetime(row['asset_addition_date'])
            current_date = pd.to_datetime(row['current_date'])
            asset_name = row['asset_name']
           
            # Fetch relevant data from end_of_day_asset_details table for the asset and time range
            query = f"SELECT * FROM end_of_day_asset_details WHERE asset_name = '{asset_name}' AND current_date BETWEEN '{asset_addition_date}' AND '{current_date}'"
            asset_details_df = pd.read_sql(query, conn)

            # Perform calculations and fill metrics columns
            asset_details_with_metrics = self.metrics(asset_details_df)
           
            # Update the row in the portfolio DataFrame with calculated metrics
            for col in asset_details_with_metrics.columns:
                if col in portfolio_df.columns:
                    portfolio_df.at[index, col] = asset_details_with_metrics.at[0, col]
       
        # Once calculations are done, update the rows in portfolio_details table with calculated metrics
        portfolio_df.to_sql('portfolio_details', con=conn, if_exists='append', index=False)
       
        # Close the database connection
        conn.close()
    def metrics(self, data):
        """
        Calculate metrics for each asset based on historical price data.

        Parameters:
        data (DataFrame): Input DataFrame containing historical price data with 'asset_name' as one of the columns.

        Returns:
        DataFrame: DataFrame with calculated metrics for each asset.
        """
        # Calculate the Sharpe ratio, Sortino ratio, Win/Loss ratio, and Max Drawdown for each asset
        grouped_data = data.groupby("asset_name")
        data["ratio_sharpe"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sharpe(x["close_price"])))
        data["ratio_sortino"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sortino(x["close_price"])))
        data["ratio_win_loss"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.win_loss_ratio(x["close_price"])))
        data["ratio_drawdown"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.max_drawdown(x["close_price"])))
       
        # Calculate monthly and yearly CAGR for each asset
        data["current_date"] = pd.to_datetime(data["current_date"])
        data_1_d =  data[data["current_date"] >= data["current_date"].max() - pd.DateOffset(days=1)]
        data_1_m = data[data["current_date"] >= data["current_date"].max() - pd.DateOffset(months=1)]
        data_3_m = data[data["current_date"] >= data["current_date"].max() - pd.DateOffset(months=3)]
        data_1_y = data[data["current_date"] >= data["current_date"].max() - pd.DateOffset(years=1)]

        data_1_d.set_index("current_date", inplace=True)
        data_1_m.set_index("current_date", inplace=True)
        data_3_m.set_index("current_date", inplace=True)
        data_1_y.set_index("current_date", inplace=True)

        data_1_d_grouped = data_1_d.groupby("asset_name")
        data_1_m_grouped = data_1_m.groupby("asset_name")
        data_3_m_grouped = data_3_m.groupby("asset_name")
        data_1_y_grouped = data_1_y.groupby("asset_name")

        # Assign CAGR and volatility values to the dataframe
        data["percentage_1_d_cagr"] = data["asset_name"].map(data_1_d_grouped.apply(lambda x: qs.stats.cagr(x["close_price"])))
        data["percentage_1_m_cagr"] = data["asset_name"].map(data_1_m_grouped.apply(lambda x: qs.stats.cagr(x["close_price"])))
        data["percentage_3_m_cagr"] = data["asset_name"].map(data_3_m_grouped.apply(lambda x: qs.stats.cagr(x["close_price"])))
        data["percentage_1_y_cagr"] = data["asset_name"].map(data_1_y_grouped.apply(lambda x: qs.stats.cagr(x["close_price"])))

        data["percentage_1_d_volatility"] = data["asset_name"].map(data_1_d_grouped.apply(lambda x: qs.stats.volatility(x["close_price"])))
        data["percentage_1_m_volatility"] = data["asset_name"].map(data_1_m_grouped.apply(lambda x: qs.stats.volatility(x["close_price"])))
        data["percentage_3_m_volatility"] = data["asset_name"].map(data_3_m_grouped.apply(lambda x: qs.stats.volatility(x["close_price"])))
        data["percentage_1_y_volatility"] = data["asset_name"].map(data_1_y_grouped.apply(lambda x: qs.stats.volatility(x["close_price"])))

        return data 